# Use Model Context Protocol (MCP) as tools with Strands Agent

## Overview
The [Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction) is an open protocol that standardizes how applications provide context to Large Language Models (LLMs). Strands AI SDK integrates with MCP to extend agent capabilities through external tools and services.

MCP enables communication between agents and MCP servers that provide additional tools. The Strands Agent SDK includes built-in support for connecting to MCP servers and using their tools.

In this example we will show you how to use MCP tools on your Strands Agent. We will use the [AWS Documentation MCP server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/) which provides tools to access AWS documentation, search for content, and get recommendations. This MCP server has 3 main features:

- **Read Documentation**: Fetch and convert AWS documentation pages to markdown format
- **Search Documentation**: Search AWS documentation using the official search API
- **Recommendations**: Get content recommendations for AWS documentation pages



## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Feature used        |MCP Tools                                          |
|Agent Structure     |Single agent architecture                          |

</div>

## Architecture

<div style="text-align:center">
    <img src="images/architecture.png" width="85%" />
</div>

## Key Features
* **Single agent architecture**: this example creates a single agent that interacts with MCP tools
* **MCP tools**: Integration of MCP tools with your agent

## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account
* Anthropic Claude 3.7 enabled on Amazon Bedrock

Let's now install the requirement packages for our Strands Agent Agent

In [ ]:
# installing pre-requisites
%pip install -r requirements.txt

### Importing dependency packages

Now let's import the dependency packages

In [1]:
import threading
import time
import os
from datetime import timedelta

from mcp import StdioServerParameters, stdio_client
from mcp.client.streamable_http import streamablehttp_client
from mcp.server import FastMCP
from strands import Agent
from strands.tools.mcp import MCPClient
from strands.models.litellm import LiteLLMModel
from strands.models.openai import OpenAIModel

### Connect to MCP server using stdio transport

[Transposts](https://modelcontextprotocol.io/specification/2025-03-26/basic/transports) in MCP provide the foundations for communication between clients and servers. It handles the underlying mechanics of how messages are sent and received. At the moment there are two standards transport implementations built-in in MCP:

- **Standard Input/Output (stdio)**: enables communication through standard input and output streams. It is particulary useful for local integrations and command-line tools
- **Streamable HTTP**: this replaces the HTTP+SSSE transport from previous protocol version. In the Streamable HTTP transport, the server operates as an independent process that can handle multiple client connections. This transport uses HTTP POST and GET requests. Server can optionally make use of Server-Sent Events (SSE) to stream multiple server messages. This permits basic MCP servers, as well as more feature-rich servers supporting streaming and server-to-client notifications and requests.

Overall, you should use stdio for building command-line tools, implementing local integrations and working with shell scripts. You should use Streamable HTTP transports when you need a flexible and efficient way for AI agents to communicate with tools and services, especially when dealing with stateless communication or when minimizing resource usage is crucial.

You can also use custom transports implementation for your specific needs. 


Let's now connect to the MCP server using stdio transport. First of all, we will use the class `MCPClient` to connect to the [AWS Documentation MCP Server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/). This server provides tools to access AWS documentation, search for content, and get recommendations.

In [2]:
# Connect to an MCP server using stdio transport
stdio_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

#### Setup agent configuration and invoke it

Next we will set our agent configuration using the tools from the `stdio_mcp_client` object we just created. To do so, we need to list the tools available in the MCP server. We can use the `list_tools_sync` method for it. 

After that, we will ask a question to our agent.

In [ ]:
# Create an agent with MCP tools
with stdio_mcp_client:
    # Get the tools from the MCP server
    tools = stdio_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(tools=tools)

    response = agent("What is Amazon Bedrock pricing model. Be concise.")

### Connect to MCP server using Streamable HTTP

Let's now connect to the MCP server using Streamable HTTP transport. First lets start a simple MCP server using Streamable HTTP transport. 

For this example we will connect to our deployed ComfyUI MCP server. The architecture will look as following

<div style="text-align:center">
    <img src="images/architecture_2.png" width="85%" />
</div>

Our ComfyUI MCP server provides the following image generation tools:
- **generate_image_with_context**: Generate images using Flux models (text-to-image and image-to-image)
- **get_comfyui_config**: Get ComfyUI configuration and available workflows

In [22]:
# We will connect to our deployed ComfyUI MCP server instead of creating a local one
# The ComfyUI MCP server provides image generation tools using Flux models
# 
# Available tools:
# - generate_image_with_context: Generate images from text prompts or transform existing images
# - get_comfyui_config: Get server configuration and available workflows
#
# Note: Make sure to replace the URL and auth token with your actual deployment values

Since we're using a deployed ComfyUI MCP server, we don't need to start a local server thread.

In [ ]:
# No need to start a local server thread since we're using the deployed ComfyUI MCP server
print("Connecting to deployed ComfyUI MCP server...")

#### Integrating Streamable HTTP client with Agent

Now lets use `streamablehttp_client` integrate this server with a simple agent. 

In [5]:
# Configure connection to your deployed ComfyUI MCP server
# Replace these values with your actual deployment details
COMFYUI_MCP_SERVER_URL = "https://your-api-gateway-url.execute-api.region.amazonaws.com/Prod/mcp"
COMFYUI_MCP_AUTH_TOKEN = "your-mcp-auth-token"

def create_comfyui_mcp_transport():
    return streamablehttp_client(
        COMFYUI_MCP_SERVER_URL,
        headers={'Authorization': f"Bearer {COMFYUI_MCP_AUTH_TOKEN}"}
    )

# Create MCP client for ComfyUI server
comfyui_mcp_client = MCPClient(create_comfyui_mcp_transport)


#### Setup agent configuration and invoke it

Next we will set our agent configuration using the tools from the `streamable_http_mcp_client` object we just created. To do so, we need to list the tools available in the MCP server. We can use the `list_tools_sync` method for it. 

After that, we will ask a question to our agent.

In [6]:
os.environ["OPENAI_API_KEY"] = "sk-mwxwvideypifnljcwlryhcnvhxxrzyueyqaqkpwvapyxhceg"
os.environ["OPENAI_BASE_URL"] = "https://api.siliconflow.cn/v1"

In [7]:
model = "azure/gpt-4.1-mini"
model = "openai/deepseek-ai/DeepSeek-V3"
litellm_model = LiteLLMModel(
    model_id=model, params={"max_tokens": 1000, "temperature": 0.7}
)

openai_compatiable_model = OpenAIModel(
    # **model_config
    client_args={
        "api_key":os.environ["OPENAI_API_KEY"],
        "base_url":"https://api.siliconflow.cn/"
    },
    model_id="deepseek-ai/DeepSeek-V3",
    max_tokens =  1000,
    temperature = 0.7,
)

In [8]:
# Connect to ComfyUI MCP server and test image generation
with comfyui_mcp_client:
    tools = comfyui_mcp_client.list_tools_sync()
    print(f"Available tools: {[tool.name for tool in tools]}")

    # Create agent with ComfyUI tools
    agent = Agent(model=openai_compatiable_model, tools=tools)

    # Test text-to-image generation
    print("\n=== Text-to-Image Generation ===")
    async for event in agent.stream_async(
        "Generate a beautiful landscape image with mountains and a lake at sunset. Use text-to-image workflow."
    ):
        if "data" in event:
            print(event["data"], end="", flush=True)
    
    print("\n\n=== Get ComfyUI Configuration ===")
    # Test getting ComfyUI configuration
    async for event in agent.stream_async("What ComfyUI workflows are available?"):
        if "data" in event:
            print(event["data"], end="", flush=True)


Tool #1: searchWebsite
以下以下是一些最新的美剧推荐（是一些最新的美剧推荐（截至2025年6月）：

1截至2025年6月）：

1. **《熊家餐馆 . **《熊家餐馆 第四季》**  
   - 第四季》**  
   - 评分：高分推荐  
  评分：高分推荐  
   - [更多详情](https - [更多详情](https://www.dealmoon.com/guide://www.dealmoon.com/guide/974237)

2. **《/974237)

2. **《一根入魂》 (一根入魂》 (Stick)**  
   - 评分Stick)**  
   - 评分：值得一看  
   - [更多：值得一看  
   - [更多详情](https://www.dealmoon详情](https://www.dealmoon.com/guide/974237)

3.com/guide/974237)

3. **《金妮与乔治亚. **《金妮与乔治亚 第三季》 (Ginny 第三季》 (Ginny & Georgia Season 3)**  
   & Georgia Season 3)**  
   - 评分：高分推荐  
   - 评分：高分推荐  
   - [更多详情](https://www - [更多详情](https://www.dealmoon.com/guide/974.dealmoon.com/guide/974237)

4. **《面237)

4. **《面面全非 第二季面全非 第二季》 (FUBAR Season 2》 (FUBAR Season 2)**  
   - 评分：高分)**  
   - 评分：高分推荐  
   - [更多详情](推荐  
   - [更多详情](https://www.dealmoon.com/https://www.dealmoon.com/guide/974237)

5. **guide/974237)

5. **《扑克脸2》**  
《扑克脸2》**  
   - 评分：经典高分  
   - 评分：经典高分  
   - [更多详情](https://   - [更多详情](https://www.dealmoon.co.uk/guideww

Session termination failed: 403


### Image-to-Image Generation with Agent\n\nFor image-to-image generation, we need to provide an input image to the agent. This demonstrates how to properly handle image inputs in Strands Agent.

In [ ]:
# Image-to-Image generation example with proper image input handling\nimport os\n\ndef test_image_to_image_generation():\n    \"\"\"Test image-to-image generation with ComfyUI MCP server\"\"\"\n    print(\"🖼️ Testing Image-to-Image Generation...\")\n    \n    try:\n        # Example image file path (replace with your actual image)\n        image_path = \"sample_image.png\"  # Replace with your image file\n        \n        if not os.path.exists(image_path):\n            print(f\"❌ Image file not found: {image_path}\")\n            print(\"Please provide a valid image file path\")\n            return\n        \n        # Read the image file\n        with open(image_path, 'rb') as image_file:\n            image_bytes = image_file.read()\n        \n        # Get image format\n        image_format = os.path.splitext(image_path)[1][1:].lower()\n        if image_format == 'jpg':\n            image_format = 'jpeg'\n        \n        # Create messages with image input\n        messages = [\n            {\n                \"role\": \"user\",\n                \"content\": [\n                    {\n                        \"image\": {\n                            \"format\": image_format,\n                            \"source\": {\n                                \"bytes\": image_bytes\n                            }\n                        }\n                    },\n                    {\n                        \"text\": \"I have uploaded an image that I want to transform.\"\n                    }\n                ]\n            }\n        ]\n        \n        print(f\"📤 Creating Agent with image input...\")\n        print(f\"   Image format: {image_format}\")\n        print(f\"   Image size: {len(image_bytes)} bytes\")\n        \n        # Connect to ComfyUI MCP server and create agent with image\n        with comfyui_mcp_client:\n            tools = comfyui_mcp_client.list_tools_sync()\n            \n            # Create agent with image messages\n            agent = Agent(\n                model=openai_compatiable_model, \n                tools=tools, \n                messages=messages  # Key: pass the image in messages\n            )\n            \n            # Now call the agent with transformation request\n            print(\"\\n=== Image-to-Image Transformation ===")\n            async for event in agent.stream_async(\n                \"Transform this image into an oil painting style using the image-to-image workflow. \"\n                \"Make it look artistic and painterly with visible brush strokes.\"\n            ):\n                if \"data\" in event:\n                    print(event[\"data\"], end=\"\", flush=True)\n            \n            print(\"\\n✅ Image-to-image generation completed!\")\n            \n    except Exception as e:\n        print(f\"❌ Image-to-image generation error: {e}\")\n        import traceback\n        traceback.print_exc()\n\n# Run the test (uncomment the line below when you have an image file)\n# test_image_to_image_generation()

### Direct Tool Invocation

While tools are typically invoked by the agent based on user requests, you can also call MCP tools directly. This can be useful for workflow scenarios where you orchestrate multiple tools together.

In [19]:
# Direct tool invocation example with ComfyUI MCP server
image_generation_params = {
    "prompt": "A serene mountain landscape with snow-capped peaks and a crystal clear lake",
    "workflow_type": "text_to_image",
    "width": 1024,
    "height": 768,
    "steps": 20,
    "cfg_scale": 7.0
}

with comfyui_mcp_client:
    # Direct tool invocation for image generation
    result = comfyui_mcp_client.call_tool_sync(
        tool_use_id="tool-img-gen", 
        name="generate_image_with_context", 
        arguments=image_generation_params
    )

    # Process the result
    if result['status'] == 'success':
        print(f"Image generation successful!")
        print(f"Result type: {result['content'][0].get('type', 'text')}")
        if 'metadata' in result['content'][0]:
            metadata = result['content'][0]['metadata']
            print(f"Generation time: {metadata.get('generation_time', 'N/A')} seconds")
            print(f"Workflow: {metadata.get('workflow_type', 'N/A')}")
    else:
        print(f"Image generation failed: {result['content'][0]['text']}")

tool execution failed: Session terminated
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 189, in call_tool_sync
    call_tool_result: MCPCallToolResult = self._invoke_on_background_thread(_call_tool_async())
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 298, in _invoke_on_background_thread
    return future.result()
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/concurrent/futures/_base.py", line 458, in result
    return self.__get_result()
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/concurrent/futures/_base.py", line 403, in __get_result
    raise self._exception
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/strands/tools/mcp/mcp_client.py", line 186, in _call_tool_async
    return await self._background_thread_session.call_tool(name

Calculation result: Tool execution failed: Session terminated


### Direct Image-to-Image Tool Invocation\n\nFor image-to-image generation via direct tool invocation, you need to provide the image as base64 encoded data in the context_image_base64 parameter.

In [ ]:
# Direct image-to-image tool invocation example\nimport base64\nimport os\n\ndef direct_image_to_image_call():\n    \"\"\"Direct tool invocation for image-to-image generation\"\"\"\n    try:\n        # Example image file path (replace with your actual image)\n        image_path = \"sample_image.png\"  # Replace with your image file\n        \n        if not os.path.exists(image_path):\n            print(f\"❌ Image file not found: {image_path}\")\n            print(\"Please provide a valid image file path\")\n            return\n        \n        # Read and encode the image\n        with open(image_path, 'rb') as image_file:\n            image_bytes = image_file.read()\n        \n        # Convert to base64 data URL format\n        image_format = os.path.splitext(image_path)[1][1:].lower()\n        if image_format == 'jpg':\n            image_format = 'jpeg'\n        \n        image_base64 = base64.b64encode(image_bytes).decode('utf-8')\n        image_data_url = f\"data:image/{image_format};base64,{image_base64}\"\n        \n        # Parameters for image-to-image generation\n        img2img_params = {\n            \"prompt\": \"Transform this image into a beautiful watercolor painting with soft, flowing colors\",\n            \"context_image_base64\": image_data_url,\n            \"workflow_type\": \"image_to_image\",\n            \"steps\": 25,\n            \"cfg_scale\": 1.0,\n            \"denoise_strength\": 0.75\n        }\n        \n        print(f\"📤 Calling image-to-image tool directly...\")\n        print(f\"   Input image size: {len(image_bytes)} bytes\")\n        print(f\"   Image format: {image_format}\")\n        \n        with comfyui_mcp_client:\n            # Direct tool invocation for image-to-image\n            result = comfyui_mcp_client.call_tool_sync(\n                tool_use_id=\"tool-img2img-direct\", \n                name=\"generate_image_with_context\", \n                arguments=img2img_params,\n                read_timeout_seconds=timedelta(seconds=120)  # 2 minutes timeout\n            )\n            \n            # Process the result\n            if result['status'] == 'success':\n                print(f\"✅ Image-to-image generation successful!\")\n                print(f\"Result type: {result['content'][0].get('type', 'text')}\")\n                if 'metadata' in result['content'][0]:\n                    metadata = result['content'][0]['metadata']\n                    print(f\"Generation time: {metadata.get('generation_time', 'N/A')} seconds\")\n                    print(f\"Workflow: {metadata.get('workflow_type', 'N/A')}\")\n                    print(f\"Denoise strength: {metadata.get('denoise_strength', 'N/A')}\")\n            else:\n                print(f\"❌ Image-to-image generation failed: {result['content'][0]['text']}\")\n                \n    except Exception as e:\n        print(f\"❌ Direct image-to-image call error: {e}\")\n        import traceback\n        traceback.print_exc()\n\n# Run the test (uncomment the line below when you have an image file)\n# direct_image_to_image_call()

You can optinally also provide `read_timeout_seconds` while calling an MCP server tool to avoid it running for too long

In [ ]:
# Example with timeout for image generation (ComfyUI can take time to generate images)
complex_image_params = {
    "prompt": "A highly detailed fantasy castle on a floating island with dragons flying around, magical aurora in the sky, photorealistic, 8k quality",
    "workflow_type": "text_to_image",
    "width": 1024,
    "height": 1024,
    "steps": 50,  # More steps for higher quality
    "cfg_scale": 8.0
}

with comfyui_mcp_client:
    try:
        result = comfyui_mcp_client.call_tool_sync(
            tool_use_id="tool-complex-img",
            name="generate_image_with_context",
            arguments=complex_image_params,
            read_timeout_seconds=timedelta(seconds=120),  # 2 minutes timeout for complex generation
        )

        if result["status"] == "error":
            print(f"Image generation failed: {result['content'][0]['text']}")
        else:
            print(f"Complex image generation succeeded!")
            if 'metadata' in result['content'][0]:
                metadata = result['content'][0]['metadata']
                print(f"Generation time: {metadata.get('generation_time', 'N/A')} seconds")
                print(f"Steps used: {metadata.get('steps', 'N/A')}")
    except Exception as e:
        print(f"Image generation timed out or failed: {str(e)}")

### Interacting with multiple MCP servers

With Strands Agents you can also interact with multiple MCP servers using the same agent and configure tools setups such as the max number of tools that can be used in parallel (`max_parallel_tools`). Let's create a new agent to show case this configuration:

<div style="text-align:center">
    <img src="images/architecture_3.png" width="85%" />
</div>

In this agent, we will use the AWS Documentation MCP server and our ComfyUI MCP server. This demonstrates how to combine different types of tools - documentation search and image generation.

First let's connect to the AWS Documentation MCP server using stdio transport and our ComfyUI server using streamable HTTP

In [7]:
# Connect to AWS Documentation MCP server using stdio transport
aws_docs_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

# We already have our ComfyUI MCP client configured above
# comfyui_mcp_client = MCPClient(create_comfyui_mcp_transport)

#### Create Agent with MCP servers

Next we will create the agent with the tools from both MCP servers

In [ ]:
# Create an agent with tools from both MCP servers
with aws_docs_mcp_client, comfyui_mcp_client:
    # Get the tools from both MCP servers
    aws_tools = aws_docs_mcp_client.list_tools_sync()
    comfyui_tools = comfyui_mcp_client.list_tools_sync()
    all_tools = aws_tools + comfyui_tools
    
    print(f"AWS Documentation tools: {[tool.name for tool in aws_tools]}")
    print(f"ComfyUI tools: {[tool.name for tool in comfyui_tools]}")

    # Create an agent with these tools
    agent = Agent(model=openai_compatiable_model, tools=all_tools, max_parallel_tools=2)

    # Test combining documentation search with image generation
    print("\n=== Multi-Server Agent Test ===")
    async for event in agent.stream_async(
        "First, search for information about Amazon Bedrock pricing. Then generate an image that represents cloud computing and AI services."
    ):
        if "data" in event:
            print(event["data"], end="", flush=True)

### Congratulations!

In this notebook you learned how to:
- Connect to MCP servers using Strands Agent with stdio and Streamable HTTP transports
- Use ComfyUI MCP server for AI image generation with Flux models
- Handle image inputs properly for image-to-image generation using the messages parameter
- Perform direct tool invocations with timeout handling
- Combine multiple MCP servers (AWS Documentation + ComfyUI) in a single agent
- Handle both text-based tools (documentation search) and media generation tools (image creation)
- Process images in both agent-based and direct tool invocation scenarios

The ComfyUI integration demonstrates how MCP can extend agent capabilities beyond text to include multimedia generation, making your agents more versatile and powerful.

## Next Steps
- **Image Generation**: Experiment with different prompts, styles, and parameters
- **Image-to-Image**: Try transformations by providing images via the messages parameter
- **Direct Tool Calls**: Use base64 encoded images in context_image_base64 parameter
- **Multi-Modal Workflows**: Combine image analysis, generation, and text processing
- **Custom ComfyUI**: Configure your server with different models and custom nodes
- **Production Deployment**: Use the provided deployment scripts for AWS Lambda
- **Error Handling**: Implement robust fallback mechanisms for image processing failures